In [554]:
import numpy as np
import pandas as pd
import re
import random
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
lemmatizer = WordNetLemmatizer()

In [555]:
df = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/dataset details.xlsx - sample questions.csv")

df = df[:190]
df.head()

,question,sql,Levels,Changes
0,How much did [customer_name] spend on [account...,select sum(debit) from master_txn_table as T1...,medium,select sum(debit) from master_txn_table as A i...
1,Did we get paid for the [product_name] this mo...,"select distinct transaction_id, AR_paid from m...",medium,NaN
2,Have we billed [customer_name] for the [produc...,select transaction_id from master_txn_table \n...,medium,NaN
3,What is the average spend per month on [accoun...,select sum(debit)/12 from master_txn_table as...,medium,NaN
4,what was our top 3 selling products and their ...,"select product_service, sum(quantity) from mas...",hard,NaN


In [556]:
df.rename(columns = {'question':'Query Pattern','sql':'Sql pattern'}, inplace = True)

In [557]:
customers = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/customers.csv")
products = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/products.csv")
vendors = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/vendors.csv")
accounts = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/chart_of_accounts.csv")
employee = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/employees.csv")
date_ranges = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/dataset details.xlsx - sample date ranges.csv")
date_ranges["sql"] = date_ranges["sql"].replace(r"\n","",regex=True)

ignore_keys = ["sample", "test", "quickbooks", "unknown", "my company", "customer", "cash", "sale", "deposit", ".", "payroll", "anonymous"]

In [558]:
def clean_names(df, var):
    flag = df[var].apply(lambda x: sum([y in x for y in ignore_keys]))
    df = df[flag == 0].reset_index(drop = True)
    return df

In [559]:
def stem_tokens(x):
    return " ".join([lemmatizer.lemmatize(y) for y in x.lower().split(" ")])

In [560]:
def get_variants(x):
    variants = [x.lower()]
    variants.append(stem_tokens(x))
    if "&" in x:
        x0 = x.split("&")[0].strip().lower()
        x1 = x.split("&")[1].strip().lower()
        variants.append(x0)
        variants.append(stem_tokens(x0))
        variants.append(x1)
        variants.append(stem_tokens(x1))
    if "|" in x:
        x0 = x.split("|")[0].strip().lower()
        x1 = x.split("|")[1].strip().lower()
        variants.append(x0)
        variants.append(stem_tokens(x0))
        variants.append(x1)
        variants.append(stem_tokens(x1))
    variants = np.unique(variants)
    return variants

In [561]:
def choose_random_attribute(filter_by_attribute):
    return random.sample(filter_by_attribute[random.sample(sorted(filter_by_attribute), 1)[0]].tolist(), 1)[0]

In [562]:
customers = clean_names(customers, "Customer name")
products = clean_names(products, "Product_Service")
vendors = clean_names(vendors, "Vendor name")
accounts = clean_names(accounts, "Account name")
employee = clean_names(employee, "Employee name")

In [563]:
customers["Variants"] = customers["Customer name"].apply(lambda x: get_variants(x))
products["Variants"] = products["Product_Service"].apply(lambda x: get_variants(x))
vendors["Variants"] = vendors["Vendor name"].apply(lambda x: get_variants(x))
accounts["Variants"] = accounts["Account name"].apply(lambda x: get_variants(x))
employee["Variants"] = employee["Employee name"].apply(lambda x: get_variants(x))
print("completed")

completed


In [564]:
# K = 50

aggregation_entity = ["total", "average", "mean", "max", "min", "first", "last", "highest", "lowest"]

intents = ["sales", "expense", "invoice", "bill", "account payable", "account receivable"]   #txn_type,

date_filter_entity = date_ranges["sample date ranges in questions"].tolist()
date_sql_entity = dict(zip(date_ranges["sample date ranges in questions"], date_ranges["sql"]))

filter_by_customer = customers.set_index("Customer name").to_dict()["Variants"]

filter_by_product = products.set_index("Product_Service").to_dict()["Variants"]

filter_by_vendor = vendors.set_index("Vendor name").to_dict()["Variants"]          #salesperson_name

filter_by_account = accounts.set_index("Account name").to_dict()["Variants"]

filter_by_employee = employee.set_index("Employee name").to_dict()["Variants"]

group_by_entity = ["account", "department", "vendor", "customer", "product"]

In [565]:
attributes_list = df["Query Pattern"].apply(lambda x:  re.findall(r'\[.*?\]', str(x) ))
attributes_list = attributes_list.apply(lambda x: [y.replace("[", "").replace("]", "") for y in x])
attributes_list = [x for y in attributes_list for x in y]
pd.Series(attributes_list).value_counts()

customer_name         69
date_filter           63
account_name          17
product_name          17
vendor_name           11
aggregation_entity    10
salesperson_name       9
transaction_type       1
groupby_entity         1
dtype: int64

In [566]:
attributes_dict = {
    "date_filter": date_filter_entity,
    "customer_name": filter_by_customer,
    "vendor_name": filter_by_vendor,
    "product_name": filter_by_product,
    "aggregation_entity": aggregation_entity,
    "account_name": filter_by_account,
    "employee_name": filter_by_employee,
    "groupby_entity": group_by_entity,
    #
    "transaction_type": intents,
    "salesperson_name": filter_by_vendor
}

In [567]:
def choose_random_attribute(attribute_dict):
    return random.sample(attribute_dict[random.sample(sorted(attribute_dict), 1)[0]].tolist(), 1)[0]

In [568]:
def get_aggregation(query):
    if any(word in query for word in ["average", "mean"]):
        return "avg"
    elif any(word in query for word in ["max", "last", "highest"]):
        return "max"
    elif any(word in query for word in ["min", "first", "lowest"]):
        return "min"
    else:
        return "sum"

In [569]:
def f1(query_pattern,d_word,is_sql=0):
    attributes = re.findall(r'\[.*?\]', str(query_pattern))
    
    # print("d_word: ",d_word)
    for a in attributes:        #[some_entity]
        
        flag = 0

        a_cleaned = re.sub(r"[\[\]]", "", a)

        new_word = d_word[0][a_cleaned]

        if is_sql == 1 and a_cleaned == "aggregation_entity" :
            new_word = get_aggregation(new_word)
        
        elif is_sql == 1 and a_cleaned == "date_filter":                          #"entity" above dict
            
            if new_word in ["yesterday","today"]:
                flag = 1

            a = '"' + a + '"'

            new_word = date_sql_entity[new_word]          # here new word is sqlquery

            if flag == 1:
                new_word = "= " + new_word

        try:
            query_pattern = query_pattern.replace(a, new_word)
        except:
            pass
        flag = 0
            
        
#     for a,w in zip(attributes,word) :
#         a_cleaned = re.sub(r"[\[\]]", "", a)
#         query_pattern = query_pattern.replace(a, d_word[a])
# #     print("$$$$",query_pattern)
    return query_pattern

In [570]:

def replace_attributes(query_pattern):
    temp = []
    d = {}
    attributes = re.findall(r'\[.*?\]', str(query_pattern))
    for a in attributes:
        a_cleaned = re.sub(r"[\[\]]", "", a)
        
        if type(attributes_dict[a_cleaned]) == list:
            select_attribute = random.sample(attributes_dict[a_cleaned], 1)[0]
        else:
            select_attribute = choose_random_attribute(attributes_dict[a_cleaned])

#         query_pattern = query_pattern.replace(a, select_attribute)
#         temp.append(select_attribute)
        d[a_cleaned]  =  select_attribute
    
    temp.append(d)
#     print(temp)
    return temp

In [571]:
df.iloc[0]["Query Pattern"]

'How much did [customer_name] spend on [account_name] in July?'

In [572]:
def simulate_questions(n=0):
    # df_sample = df[~df["Query Pattern"].apply(lambda x: "salesperson_name" in x)].reset_index(drop=True)
    df_sample = df.copy()
    
    # df_simulated = df_sample.loc[random.choices(range(len(df_sample)), k=n)].reset_index(drop=True)
    df_simulated = df_sample.loc[np.arange(len(df_sample))].reset_index(drop=True)
    
#     df_simulated["Query"] = df_simulated["Query Pattern"].apply(lambda x: replace_attributes(x))
    
#     df_simulated["Sql pattern new"] = df_simulated["Sql pattern"].apply(lambda x: replace_attributes(str(x)))
#     df_simulated.loc[df_simulated["Aggregation"].isna(), "Aggregation"] = df_simulated[df_simulated["Aggregation"].isna()]["Query"].apply(lambda x: get_aggregation(x))

    ls1 = df_simulated["Query Pattern"].apply(lambda x: replace_attributes(x)).to_list()
    # print(ls1)
    df_simulated["Query"] = np.nan
    df_simulated["Sql pattern new"] = np.nan
    

    for i in range(len(ls1)):
        
        print("asdasdas  ",i,ls1[i])
        print(df_simulated.loc[i].to_list())
        print()

        
        df_simulated.loc[[i],["Query"] ] = f1(df_simulated.iloc[i]["Query Pattern"],ls1[i])
        df_simulated.loc[[i],["Sql pattern new"] ] = f1(df_simulated.iloc[i]["Sql pattern"],ls1[i],is_sql=1)

    
    
    return df_simulated

In [573]:
df_simulated = simulate_questions(10)

asdasdas   0 [{'customer_name': "pye's cakes", 'account_name': 'gas and electric'}]
['How much did [customer_name] spend on [account_name] in July?', 'select sum(debit) from master_txn_table  as T1 join chart_of_accounts as T2 on T1.account = T2.account_name where T1.customers = "[customer_name]" and instr(account,"[account_name]") and strftime("%m", transaction_date) = 7 and account_type in (\'Expense\',\'Other Expense\')', 'medium', 'select sum(debit) from master_txn_table as A inner join chart_of_accounts as B on A.account = B.account_name\nwhere customer = "[customer_name]" and account.Contains("[account_name]") and month(transaction_date) = 7 and account_type in (\'Expense\',\'Other Expense\')', nan, nan]

asdasdas   1 [{'product_name': 'gardening'}]
['Did we get paid for the [product_name] this month?', 'select distinct transaction_id, AR_paid from master_txn_table \nwhere product_service = "[product_name]" and strftime("%m", transaction_date) = strftime("%m", current_timestamp);

In [574]:
df_simulated["Sql pattern"] = df_simulated["Sql pattern"].replace(r"\n","",regex=True)
df_simulated["Sql pattern new"] = df_simulated["Sql pattern new"].replace(r"\n","",regex=True)

In [575]:
df_simulated[["Sql pattern","Sql pattern new"]].head()

,Sql pattern,Sql pattern new
0,select sum(debit) from master_txn_table as T1...,select sum(debit) from master_txn_table as T1...
1,"select distinct transaction_id, AR_paid from m...","select distinct transaction_id, AR_paid from m..."
2,select transaction_id from master_txn_table wh...,select transaction_id from master_txn_table wh...
3,select sum(debit)/12 from master_txn_table as...,select sum(debit)/12 from master_txn_table as...
4,"select product_service, sum(quantity) from mas...","select product_service, sum(quantity) from mas..."


In [576]:
final_df = df_simulated[['Query','Sql pattern new']]
final_df.head()

,Query,Sql pattern new
0,How much did pye's cakes spend on gas and elec...,select sum(debit) from master_txn_table as T1...
1,Did we get paid for the gardening this month?,"select distinct transaction_id, AR_paid from m..."
2,Have we billed geeta kalapatapu for the servic...,select transaction_id from master_txn_table wh...
3,What is the average spend per month on savings...,select sum(debit)/12 from master_txn_table as...
4,what was our top 3 selling products and their ...,"select product_service, sum(quantity) from mas..."


In [577]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('max_colwidth', -1)

In [578]:
final_df["Sql pattern new"]

0      select sum(debit) from master_txn_table  as T1...
1      select distinct transaction_id, AR_paid from m...
2      select transaction_id from master_txn_table wh...
3      select sum(debit)/12 from master_txn_table  as...
4      select product_service, sum(quantity) from mas...
                             ...                        
185    select min(credit) from master_txn_table where...
186    select avg(credit) from master_txn_table where...
187    select transaction_date from master_txn_table ...
188    select count(distinct transaction_id) from mas...
189    select count(distinct transaction_id) from mas...
Name: Sql pattern new, Length: 190, dtype: object

In [579]:
# for i in final_df["Sql pattern new"].to_list():
#     print(i)

In [580]:
temp = []

#connecting to DB
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("/data/unified_parser_text_to_sql/spider/preprocess/f1/accounting/accounting.sqlite")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
# for row in cur.execute('SELECT * FROM species;'):
#     print(row)

j=2
for i in final_df["Sql pattern new"].to_list():
    # print(j)
    # print(i)
    # print()
    try:
        cur.execute(str(i))
        res = cur.fetchone()
        # print(res)
    except:
        temp.append([j,i])
        # print(j)
        # print(i)
        # print()
    
    j+=1

# Be sure to close the connection
con.close()


In [581]:
print(final_df["Sql pattern new"][0])

select sum(debit) from master_txn_table  as T1 join chart_of_accounts as T2 on T1.account = T2.account_name where T1.customers = "pye's cakes" and instr(account,"gas and electric") and strftime("%m", transaction_date) = 7 and account_type in ('Expense','Other Expense')


In [582]:
temp[0]

[23,
 'select balance from chart_of_accounts where customers = "geeta kalapatapu" and account_name = \'checking\'']

In [583]:

for i in temp:
    if i[0]< 141:
        continue
    print(i[0], end = ' ')
    print(i[1])

154 select count(distinct transaction_id) from master_txn_table where transaction_type = 'sales receipt' and misc.salesperson = 'salesperson' and transaction_date BETWEEN date( current_date, "start of month") AND date( current_date) 
155 select count(distinct transaction_id) from master_txn_table where transaction_type = 'sales receipt' and misc.salesperson = 'salesperson' and transaction_date BETWEEN date(current_date) AND date(current_date) 
156 select count(distinct transaction_id) from master_txn_table where transaction_type = 'sales receipt' and misc.salesperson = 'salesperson' and transaction_date BETWEEN date( current_date, "start of month") AND date( current_date) 
157 select count(distinct transaction_id) from master_txn_table where transaction_type = 'sales receipt' and misc.salesperson = 'salesperson' and transaction_date BETWEEN date( current_date, "start of month") AND date( current_date) 
158 select count(distinct transaction_id) from master_txn_table where transaction_ty

In [584]:
final_df.iloc[189].to_list()

['Number of invoices created for decks and patios in Last quarter?',
 'select count(distinct transaction_id) from master_txn_table where transaction_type = \'invoice\' and instr(account,"decks and patios") and transaction_date BETWEEN date(current_date) AND date(current_date) ']

In [585]:
len(temp)

23